In [1]:
import importlib.resources as ires
from autointent.context.utils import load_data

def get_dataset_path():
    return ires.files("tests.assets.data").joinpath("clinc_subset.json")


def get_dataset():
    return load_data(get_dataset_path())

In [2]:
dataset = get_dataset()
len(dataset.utterances), len(dataset.intents)

(18, 3)

In [3]:
from autointent.context.data_handler import DataHandler

data_handler = DataHandler(dataset)

In [4]:
len(data_handler.utterances_train), len(data_handler.utterances_test), len(data_handler.oos_utterances)

(11, 4, 3)

In [5]:
from autointent.modules.scoring import KNNScorer

knn_params = {
    "k": 3,
    "weights": "distance",
    "embedder_name": "sergeyzh/rubert-tiny-turbo",
    "db_dir": "db_dir",
}
scorer = KNNScorer(**knn_params)

scorer.fit(data_handler.utterances_train, data_handler.labels_train)
scores = scorer.predict(data_handler.utterances_test + data_handler.oos_utterances)

In [6]:
scores

array([[0.        , 1.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [1.        , 0.        , 0.        ],
       [0.        , 0.7688683 , 0.2311317 ],
       [0.32017301, 0.67982699, 0.        ],
       [1.        , 0.        , 0.        ]])

In [7]:
labels = data_handler.labels_test + [-1] * len(data_handler.oos_utterances)

In [8]:
labels

[1, 0, 2, 0, -1, -1, -1]

In [9]:
from autointent.modules.prediction import ThresholdPredictor
import numpy as np

predictor = ThresholdPredictor(np.array([0.5, 0.5, 0.5]))
predictor.fit(scores, labels)

In [ ]:
example_scores = np.array([[0.1, 0.9, 0], [0.8, 0, 0.2], [0, 0.3, 0.7]])
predictions = predictor.predict(example_scores)

ValueError: Provided scores number don't match with number of classes which predictor was trained on.